# TP 2-3 : Programmation dynamique

### Initialisation (à faire une seule fois)

### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

###  fonction de créaction de tableau 


In [2]:
    # fonction: CreatTableau
    # Les paramètres d'entrée: liste des couts(prix), poids des objets dans l'odre décroissant des prix,
    #              la capacité maximale du sac 
    # Valeur duretour : le tableau des prix maximaux en fonction du poids maximal du sac et du nombre d'objets 
    
    function CreatTableau(cout, poids, capacite)
    lcout = length(cout)
    tableau = [(0,[]) for i=1:(lcout), j=1:(capacite+1)]
    
    for j in 1:(capacite+1)                 #pour initialiser la première ligne du tableau
        if j-1 >= poids[1]
            (p,t) = tableau[1,j]
            tableau[1,j] = (cout[1],[1])
        end
    end
    
    
    for i in 2:(lcout)                  
        for j in 2:(capacite+1)
            if (j-poids[i]) > 0               #On applique la relation du cours
                (p1,t1) = tableau[(i-1),j]
                (p2,t2) = tableau[(i-1),(j-poids[i])]
                if p2 + cout[i] > p1
                    t3 = copy(t2)
                    t3 = push!(t3,i)
                    tableau[i,j] = (p2 + cout[i], t3)
                else 
                    tableau[i,j] = (p1,t1)
                end
            else
                (p1,t1) = tableau[(i-1),j]
                tableau[i,j] = (p1,t1)
            end
        end
    end
    return tableau
end

#Tout d'abord on définit la fonction createTableau qui s'occupe de la création d'un tableau 
#des prix maximaux en fonction du poids maximal du sac et du nombre d'objets 

#t3: liste ou on met l'élément qu'on ajouté au sac pour qu'il aura cette capacité



   

CreatTableau (generic function with 1 method)

In [1]:
### Expliquons le fonctionnement du programme ci dessus

Le programme prend en entrée trois tableaux : 'cout', 'poids' et 'capacite'. Le tableau cout contient les coûts de chaque 
objet, le tableau poids contient les poids de chaque objet et la variable capacite représente la capacité maximale du 
sac à dos.

Aprés on crée un tableau 'tableau' qui contient des tuples de la forme (prix, indices). Le tuple (prix, indices) à 
l'indice (i,j) du tableau 'tableau' représente le prix maximal que l'on peut obtenir en prenant des objets d'indices compris 
entre 1 et i, e


t en respectant une contrainte de poids maximale de j.

Le programme commence par initialiser la première ligne du tableau tableau en vérifiant si le poids de l'objet 1 est inférieur 
à la colonne actuelle du tableau. Si c'est le cas, alors le prix maximal est mis à jour avec le coût de l'objet 1 et l'indice 
de l'objet est ajouté à la liste des indices.

Ensuite, le programme parcourt chaque ligne et chaque colonne du tableau tableau en utilisant les boucles for et met à jour
le tuple (prix, indices) en utilisant la relation du cours vue en classe. Si le poids de l'objet actuel plus le poids 
maximal déjà pris est inférieur à la colonne actuelle du tableau, alors le tuple (prix, indices) est mis à jour avec 
le coût de l'objet actuel et l'indice de l'objet est ajouté à la liste des indices. Sinon, le tuple (prix, indices) est 
laissé inchangé.

Enfin, le programme retourne le tableau 'tableau' qui contient les tuples (prix, indices) pour chaque contrainte de poids 
maximale.

###  fonction de recherche de solution


In [3]:
# fonction : FoundSolution
# parametres d'entrée: le tableau retourné par la fonction CreatTableau
#              le nombre d'objets concidérés au départ 
#              la capacité maximale du sac 
# retour : le cout maximum correspôndant à la capacité maximale du sac, 
#          les indices dans l'ordre décroissant de prix des objets dans le sac 

function FoundSolution(tableau, lcout, capacite)
    coutMax = 0
    indMax = []
    for i in 1:lcout
        (t,ind) = tableau[i,capacite+1] 
        if t > coutMax
            coutMax = t
            indMax = ind
        end
    end
    return coutMax, indMax
end

FoundSolution (generic function with 1 method)

In [ ]:
###Expliquons le fonctionnement du programme ci dessus

La fonction 'FoundSolution' prend en entrée un tableau de résultats (retourné par la fonction CreatTableau), le nombre 
d'objets concidérés au départ et la capacité maximale du sac à dos.

La fonction parcourt chaque ligne du tableau et cherche le tuple (prix, indices) qui a le plus grand prix. Elle met à 
jour la variable 'coutMax' avec le prix maximal trouvé et la variable 'indMax' avec la liste des indices correspondant à ce 
prix maximal.

Enfin, la fonction retourne le tuple (coutMax, indMax) qui contient le coût maximal trouvé et la liste des indices des 
objets qui ont été sélectionnés pour atteindre ce coût maximal.

### Boucle principale : résoudre le problème par programmation dynamique

In [7]:

function SolveKnapInstance(filename)

        cout, poids, capacite = readKnaptxtInstance(filename)
        
        indicesOrdonne = sortperm(cout) #ordonner le vecteur cout de manière décroissante
        indicesOrdonne = reverse(indicesOrdonne)
        poidsOrdonne = poids[indicesOrdonne]        
        coutOrdonne = cout[indicesOrdonne]
        
        tab = CreatTableau(coutOrdonne, poidsOrdonne, capacite) #On appelle la fonction createTableau aprés avoir ordonné de façon décroissante le poids des objets
        
        MeilleurGain, indMax = FoundSolution(tab, length(cout), capacite)
        
        MeilleurSol = [0 for i=1:length(cout)]
        ind = indicesOrdonne[indMax]
        for k in ind
            MeilleurSol[k] = 1
        end
        
        return MeilleurGain, MeilleurSol
    end


SolveKnapInstance (generic function with 1 method)

In [ ]:
###Expliquons le fonctionnement du programme ci dessus:
La fonction 'SolveKnapInstance' est utilisée pour résoudre une instance particulière du problème du sac à dos en utilisant 
l'algorithme de programmation dynamique. Elle prend en entrée un nom de fichier qui contient les informations sur les 
coûts, les poids et la contrainte de poids maximale pour l'instance du problème du sac à dos.

La fonction commence par lire ces informations à partir du fichier en appelant la fonction readKnaptxtInstance et en 
stockant les résultats dans les variables 'cout', 'poids' et 'capacite'.

Ensuite, la fonction ordonne les vecteurs cout et poids de manière décroissante en utilisant la fonction 'sortperm' et en 
les inversant avec reverse. Ces vecteurs ordonnés sont stockés dans les variables coutOrdonne et poidsOrdonne.

La fonction appelle ensuite la fonction 'CreatTableau' en passant en entrée les vecteurs ordonnés 'coutOrdonne' et 
'poidsOrdonne', ainsi que la contrainte de poids maximale 'capacite'. Le tableau de résultats retourné par la fonction 
'CreatTableau' est stocké dans la variable tab.

Ensuite, la fonction appelle la fonction FoundSolution en passant en entrée le tableau de résultats tab, le nombre 
d'objets concidérés au départ et la contrainte de poids maximale capacite. La fonction FoundSolution retourne le coût 
maximal trouvé et la liste des indices des objets qui ont été sélectionnés pour atteindre ce coût maximal, qui sont 
stockés dans les variables 'MeilleurGain' et 'indMax'.

Enfin, la fonction construit la solution finale en mettant à jour la liste MeilleurSol avec un 1 pour chaque objet qui a 
été sélectionné et 0 pour les autres. La fonction retourne le coût maximal MeilleurGain et la liste des objets 
sélectionnés MeilleurSol.

### Affichage du résultat final

In [8]:
fold = "instancesETU/KNAPnewformat/test.opb.txt"

MeilleurGain, MeilleurSol = SolveKnapInstance(fold)
println("\n******\n\nOptimal value = ", MeilleurGain, "\n\nOptimal x=", MeilleurSol)
     


******

Optimal value = 65

Optimal x=[0, 1, 0, 1]


In [9]:
fold = "instancesETU/KNAPnewformat/circle/knapPI_16_100_1000_5_
-8569.opb.txt"

MeilleurGain, MeilleurSol = SolveKnapInstance(fold)
println("\n******\n\nOptimal value = ", MeilleurGain, "\n\nOptimal x=", MeilleurSol)
     


******

Optimal value = 12593

Optimal x=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [14]:
fold = "instancesETU/KNAPnewformat/circle/knapPI_16_100_1000_2_-3898.opb.txt"

MeilleurGain, MeilleurSol = SolveKnapInstance(fold)
println("\n******\n\nOptimal value = ", MeilleurGain, "\n\nOptimal x=", MeilleurSol)
     


******

Optimal value = 4953

Optimal x=[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


In [ ]:
###Comparaison avec branch-and-bound: